pip install tensorflow matplotlib


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os

In [ ]:
train_dir = 'dataset/train'
val_dir   = 'dataset/validation'
